# My Playground for Neural Network

In [358]:
from utils import *
from stratified import *
from run import *
from neuralnetwork import *

In [359]:
def importhousedata():
    house = importfile('hw3_house_votes_84.csv', ',')
    housecategory = {}
    for i in house[0]:
        housecategory[i] = 'categorical'
    housecategory["class"] = 'class'
    housedata = np.array(house[1:]).astype(float)
    return housedata, housecategory
def importwinedata():
    wine = importfile('hw3_wine.csv', '\t')
    winecategory = {}
    for i in wine[0]:
        winecategory[i] = 'numerical'
    winecategory["# class"] = 'class'
    winedata = np.array(wine[1:]).astype(float)
    return winedata, winecategory
def importcmcdata():
    cmc = importfile('cmc.data', ',')
    cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
    "Husband's education":"categorical","Number of children ever born":"numerical",
    "Wife's religion":"binary","Wife's now working?":"binary",
    "Husband's occupation":"categorical","Standard-of-living index":"categorical",
    "Media exposure":"binary","Contraceptive method used":"class"}
    cmcdata = np.array(cmc).astype(int)
    return cmcdata, cmccategory
def importcancerdata():
    cancer = importfile('hw3_cancer.csv', '\t')
    cancercategory = {}
    for i in cancer[0]:
        cancercategory[i] = 'numerical'
    cancercategory["Class"] = 'class'
    cancerdata = np.array(cancer[1:]).astype(float)
    return cancerdata, cancercategory

housedata, housecategory = importhousedata()
winedata, winecategory = importwinedata()
cmcdata,cmccategory = importcmcdata()
cancerdata, cancercategory = importcancerdata()
example_category = {'ece': 'numerical','cs': 'categorical','phy': 'categorical','math': 'categorical','musi': 'categorical','grade': 'class'}
#example_data = np.array([[100,0,1,1,0,0],[123,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2]])
example_data = np.array([[10,0,1,1,0,0],[23,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2],[122,0,1,1,0,0],[124,1,0,1,1,1],[44,1,0,0,0,2],[40,1,0,1,0,3],[73,0,1,0,0,2]])

In [360]:
ohe_house_data, ohe_house_category = onehotencoder(housedata, housecategory)
ohe_wine_data, ohe_wine_category = onehotencoder(winedata,winecategory)
ohe_cmc_data, ohe_cmc_category = onehotencoder(cmcdata,cmccategory)
ohe_example_data, ohe_example_category = onehotencoder(example_data,example_category)
ohe_cancer_data, ohe_cancer_category = onehotencoder(cancerdata,cancercategory)

In [361]:
def g(x): # sigmoid function
    return np.exp(x) / (1 + np.exp(x))

In [362]:
def costfunction(expected_output, actual_output):
    j = -np.dot(expected_output,np.log(actual_output) + np.dot((1 - expected_output),np.log(1 - actual_output)))
    return j

In [363]:
def sumofweights(listofweights,bias=True): # computes the square of all weights of the network and sum them up
    sum = 0
    for weight in listofweights:
        if bias:
            sum += np.sum(np.square(weight[1:]))
        else:
            sum += np.sum(np.square(weight))
    return sum

In [364]:
# Forward propagation vectorized
def neural_network(normed_hotted_data,ohe_category,weights_list,biasterm=True, minibatchk = 15, lambda_reg = 0.1):
    normed_ohe_copy = normed_hotted_data.copy()
    np.random.shuffle(normed_ohe_copy)
    splitted = np.array_split(normed_ohe_copy, minibatchk)
    
    inputcategory, outputcategory = [],[]
    inputindex, outputindex = [],[]
    n = 0
    for i in ohe_category:
        if ohe_category[i] != 'class_numerical':
            inputcategory.append(i) # name of the input category
            inputindex.append(n) # index of the input category
        else:
            outputcategory.append(i) # name of the output category
            outputindex.append(n) # index of the output category
        n += 1
    
    b = 1 if biasterm else 0
    
    for onebatch in splitted:
        onebatch = onebatch.T
        input_data = onebatch[inputindex].T
        output_data = onebatch[outputindex].T
        # input_data_mean = onebatch[inputindex].mean(axis=1)
        output_data = onebatch[outputindex].T

        # forward propagation
        instance_index = 0
        j = 0

        for one_instance in input_data:
            one_instance = np.append(1,one_instance) if b == 1 else one_instance
            current_layer_a = one_instance # input layer is the current layer
            current_layer_index = 0
            output_real = output_data[instance_index]

            for theta in weights_list:
                z = np.dot(theta,current_layer_a)
                a = g(z)
                current_layer_a = np.append(1,a) if (b == 1) and (current_layer_index+1 != len(weights_list)) else a
                current_layer_index += 1

            output_predict = current_layer_a # the last attribute is the output for this batch.
            instance_index += 1
            j += costfunction(output_real,output_predict)
        
        j /= (instance_index+1)
        s = sumofweights(weights_list,bias=b)*lambda_reg/(2*(instance_index+1))
        j += s # total cose with regularization

     
    if b==1: ## Place Holder
        c = 1

    return input_data

In [365]:
def back_propagate(real_output, expected_output, weights_list, biasterm=True):
    deltalist = []
    delta_layer_l = expected_output - real_output
    print('back propagate',delta_layer_l)
    return

In [366]:
def sumofweights(listofweights,bias=True): # computes the square of all weights of the network and sum them up
    sum = 0
    for weight in listofweights:
        if bias:
            sum += np.sum(np.square(weight.T[1:].T))
        else:
            sum += np.sum(np.square(weight))
    
    return sum

In [367]:
layerdataparameter = [4,5,4] # [First layer, Second layer, Third layer], not including the bias and input & output layer
initweights = initialize_weights(ohe_house_category,layerdataparameter)
print([len(i) for i in initweights])
kk = neural_network(ohe_house_data,ohe_house_category,initweights)
# print(kk)

[4, 5, 4, 2]
1.7655339350731531
2.1523192741465853
2.2294287474082144
1.8447580175765643
2.230707386499633
2.2288875055832826
2.0744600697562086
1.999197056881636
2.0769671036231068
2.079224709115341
1.7664612599795986
1.9968171343594718
2.151954549256049
2.0754627776004146
2.228678761273375


In [368]:
k = 5
wok = np.array([[1,2],[3,4],[5,6],[7,8],[9,10],[11,12],[13,14],[15,16]])
whaha = len(wok)
print(whaha)
print(wok.shape[0])
random_indices = np.random.choice(whaha, size=k, replace=False)
random_rows = wok[random_indices]
print(random_rows)

8
8
[[11 12]
 [15 16]
 [ 7  8]
 [ 1  2]
 [13 14]]


In [369]:
layernow = 3
layernext = 2
init_weight = np.random.rand(layernext,layernow) * 2 - 1
a_now = np.random.rand(layernow,1)
print(init_weight)
print(a_now)
c = np.dot(init_weight,a_now)
print(c)

[[ 0.6748304  -0.03498296 -0.70587263]
 [-0.43734238 -0.19097771 -0.50966684]]
[[0.00981242]
 [0.68912713]
 [0.77356676]]
[[-0.56352559]
 [-0.53016063]]
